# 5.

hyperparameter tuning with wandb on GBM

In [45]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score, recall_score, roc_curve, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

import wandb
from wandb.sklearn import plot_precision_recall, plot_feature_importances
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc

import optuna

In [2]:
split_date = '2022-10-01'
F = pd.read_csv('features.csv')

X = F[['eps_est', 'eps_actual', 'eps_diff',
       'eps_diff_pct', 
       #'tot_assets', 'common_stock', 'tot_cap', 'tot_debt',
       #'net_debt', 'ord_shares', 'tot_reve', 'gross_profit', 'op_expense',
       #'op_income', 'avg_shares', 'prev_eps_actual', 'prev_eps_diff',
       #'expense_to_income',
       #'eps_q_change', 'eps_q_change_pct',
       #'assets_to_profit', 'profit_to_reve',
       'debt_to_debt', 'debt_to_cap', 
       'on_earnings_close', 'on_earnings_volume', 
       'market_cap']]
y = F[['tgt']]
INFO = F[['ticker', 'earnings_date','history_start_date', 'history_end_date', 'price_min', 'price_max','tgt']]

ndx_train = INFO[INFO.earnings_date< split_date].index.to_list()
ndx_test  = INFO[INFO.earnings_date>=split_date].index.to_list()

X_train,    X_test    = X.iloc[ndx_train],    X.iloc[ndx_test]
y_train,    y_test    = y.iloc[ndx_train],    y.iloc[ndx_test]
INFO_train, INFO_test = INFO.iloc[ndx_train], INFO.iloc[ndx_test]

## tuning

In [17]:
config = {
    'learning_rate' : 0.05, 
    'n_estimators' : 50, 
    'subsample' : .9,
    'random_state':7
}
labels = ['loose','gain']

In [87]:
def objective(trial):

    max_depth = trial.suggest_int('max_depth', 1, 3, log=False)
    lr = trial.suggest_float('lr', 0.1, 1.0, log=False)
    n_est = trial.suggest_int('n_est', 10, 200, log=True)
    subsample = trial.suggest_float('subsample', 0.5, 1.0, log=False)
    loss = trial.suggest_categorical('loss',['deviance', 'exponential'])
    criterion = trial.suggest_categorical('criterion',['friedman_mse', 'squared_error'])

    THRESHOLD = trial.suggest_float('THRESHOLD', .6, .8, log=False)
    
    classifier_obj = GradientBoostingClassifier(
        max_depth=max_depth, 
        learning_rate=lr,
        n_estimators=n_est,
        subsample=subsample,
        loss=loss,
        criterion=criterion
        )

    cv_res = cross_validate(classifier_obj, X_train, y_train.tgt, 
        cv=8, 
        #return_train_score=True,
        scoring = {
            'accuracy':make_scorer(accuracy_score),
            'precision':make_scorer(precision_score),
            'npositive':make_scorer(lambda yt,yp: yp.sum()),
            'myscore':my_scorer,
        }
    )
    #score = cross_val_score(classifier_obj, X_train, y_train['tgt'], n_jobs=-1, cv=5)
    #accuracy = score.mean()

    precision = cv_res['test_precision'].mean()
    accuracy  = cv_res['test_accuracy'].mean()
    npositive = cv_res['test_npositive'].mean()
    myscore   = cv_res['test_myscore'].mean()
    print('precision',precision,'npositive',npositive)
    return myscore

study = optuna.create_study(
    direction='maximize',
    storage="sqlite:///db.sqlite3", # https://github.com/optuna/optuna-dashboard
    # study_name="trial-study"
    )
study.optimize(objective, n_trials=100)
print(study.best_params, "best score:", study.best_value)

[I 2023-01-24 14:53:44,035] A new study created in RDB with name: no-name-5edcd759-4ca4-40e3-a29a-3f38ddc42d64
[I 2023-01-24 14:53:45,558] Trial 0 finished with value: 0.5553421841568393 and parameters: {'max_depth': 2, 'lr': 0.707590544407899, 'n_est': 25, 'subsample': 0.6861583185794491, 'loss': 'deviance', 'criterion': 'friedman_mse', 'THRESHOLD': 0.6771109591260235}. Best is trial 0 with value: 0.5553421841568393.


precision 0.4992354939355578 npositive 73.5


[I 2023-01-24 14:53:49,252] Trial 1 finished with value: 0.545080408245454 and parameters: {'max_depth': 2, 'lr': 0.18994422285193782, 'n_est': 164, 'subsample': 0.995646146883112, 'loss': 'deviance', 'criterion': 'squared_error', 'THRESHOLD': 0.6056933291767496}. Best is trial 0 with value: 0.5553421841568393.


precision 0.5167955975662376 npositive 69.875


[I 2023-01-24 14:53:50,652] Trial 2 finished with value: 0.5900568181818182 and parameters: {'max_depth': 1, 'lr': 0.7998347451249926, 'n_est': 21, 'subsample': 0.878205909313384, 'loss': 'deviance', 'criterion': 'friedman_mse', 'THRESHOLD': 0.6957091935392445}. Best is trial 2 with value: 0.5900568181818182.


...

precision 0.5176324410596927 npositive 66.625


[I 2023-01-24 14:56:00,649] Trial 98 finished with value: 0.5874074454956808 and parameters: {'max_depth': 1, 'lr': 0.9787919468008368, 'n_est': 21, 'subsample': 0.5372844559597795, 'loss': 'deviance', 'criterion': 'squared_error', 'THRESHOLD': 0.6602867373653583}. Best is trial 78 with value: 0.7088760198135198.


precision 0.5281796229898245 npositive 69.5


[I 2023-01-24 14:56:02,144] Trial 99 finished with value: 0.5574723877143442 and parameters: {'max_depth': 2, 'lr': 0.7439926235263933, 'n_est': 27, 'subsample': 0.5119710451661837, 'loss': 'deviance', 'criterion': 'squared_error', 'THRESHOLD': 0.6514992051200188}. Best is trial 78 with value: 0.7088760198135198.


precision 0.5224621302793611 npositive 73.25
{'THRESHOLD': 0.7070798477604443, 'criterion': 'squared_error', 'loss': 'deviance', 'lr': 0.8316516913375693, 'max_depth': 1, 'n_est': 13, 'subsample': 0.5683471085944433} best score: 0.7088760198135198


In [9]:
model = GradientBoostingClassifier(**config)
model.fit(X_train, y_train['tgt'])
model_params = model.get_params()

# get predictions
y_pred = model.predict(X_test)
y_probas = model.predict_proba(X_test)
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
# precision opt: 
# {'criterion': 'squared_error', 'loss': 'exponential', 'lr': 0.1830342144130779, 'max_depth': 1, 'n_est': 48, 'subsample': 0.7333617656935432} best score: 0.577118572534203
# accuracy opt:
# {'criterion': 'squared_error', 'loss': 'exponential', 'lr': 0.18664109568913484, 'max_depth': 1, 'n_est': 83, 'subsample': 0.5919092968415383} best score: 0.5883931753385603
# myscore opt:
# {'THRESHOLD': 0.617040203257516, 'criterion': 'squared_error', 'loss': 'exponential', 'lr': 0.15987365290248487, 'max_depth': 1, 'n_est': 75, 'subsample': 0.9266465385304337} 
# best score: 0.5706290505769389

In [85]:
THRESHOLD=0.7 # this is a global param, modeified by the optuna
def my_scorer(clf, _X_test, _y_test):
    y_pred_proba = clf.predict_proba(_X_test)
    y_pred = (y_pred_proba[:,1]>THRESHOLD).astype(int)

    cm = confusion_matrix(_y_test, y_pred)
    p  = precision_score(_y_test, y_pred)
    total_pos = cm[:,1].sum()
    min_trades_in_period = 8 #8
    quantity_score = min(total_pos/min_trades_in_period, 1.0) # have at least min_trades otherwise, fall hyperbolically
    SCORE = p * quantity_score # score to maximize: tradeoff between precision and quantity
    return SCORE

In [86]:
cross_validate(model, X_train, y_train.tgt, 
    cv=8, 
    #return_train_score=True,
    scoring = {
        # 'accuracy':make_scorer(accuracy_score),
        # 'precision':make_scorer(precision_score),
        # 'npositive':make_scorer(lambda yt,yp: yp.sum()),
        'myscore':my_scorer,
    }
)

{'fit_time': array([0.24406624, 0.18196225, 0.17646861, 0.16819596, 0.17685556,
        0.16884923, 0.13920975, 0.1377418 ]),
 'score_time': array([0.00548887, 0.00468087, 0.00452447, 0.00423455, 0.0041604 ,
        0.0042944 , 0.00276542, 0.00295949]),
 'test_myscore': array([0.25 , 0.125, 0.5  , 0.375, 0.25 , 0.375, 0.125, 0.125])}